In [ ]:
import pandas as pd 
import numpy

bdgd = pd.read_excel('./Bases/Ucs_UCAT.xlsx',
dtype={'COD_ID':str})

In [ ]:
def ibge_completo():
    
    municipios_ibge = pd.read_excel('./Bases/RELATORIO_DTB_BRASIL_DISTRITO.xls',
    usecols=['Código Município Completo','UF','Nome_Município'])

    ibge_estados = pd.read_excel('./Bases/Tabela_Estados_IBGE.xlsx')

    #Mudando Nome da Coluna do IBGE e Estados
    municipios_ibge\
        .rename(columns={'Código Município Completo':'MUN',
        'Nome_Município':'MUNICIPIO','UF':'COD_UF'},inplace=True)

    ibge_estados\
        .rename(columns={'Código da UF':'COD_UF','Estado':"ESTADO"},inplace=True)


    #Deixando Municipios e Estado em extenso em letra maiscula
    municipios_ibge.MUNICIPIO = municipios_ibge\
        .MUNICIPIO.str.upper()

    ibge_estados.ESTADO = ibge_estados\
        .ESTADO.str.upper()

    #Cruzando com IBGE com siglas de Municipio 
    ibge_completo = municipios_ibge.merge(ibge_estados,on='COD_UF')

    #Removendo linhas duplicadas do datafrmae
    ibge_completo.drop_duplicates(inplace=True)

    return ibge_completo
    
ibge_completo = ibge_completo()

In [ ]:
print(f"bdgd{bdgd.shape} | ibge {ibge_completo.shape}")

In [ ]:
#Cruzando codigo_mun bdgd com ibge_completo
dataframe_completo = bdgd.merge(ibge_completo,on='MUN',
how='left',copy=True)  

print(f"Cruzamento{dataframe_completo.shape} ")

In [ ]:
df_consolidado = dataframe_completo[:] ## ** Melhorar código depois

In [ ]:
#Criando Coluna de Demanda agrupando Ponta e Fora ponta
for x in range(1,13):
    
    if x < 10:
        df_consolidado[f'DEM_{x}'] = (df_consolidado[f'DEM_P_0{x}'] + df_consolidado[f'DEM_F_0{x}'])
        
    else:
        df_consolidado[f'DEM_{x}'] = (df_consolidado[f'DEM_P_{x}'] + df_consolidado[f'DEM_F_{x}'])       

In [ ]:
#Criando Coluna de Energia agrupando Ponta e Fora ponta
for x in range(1,13):
    
    if x < 10:
        df_consolidado[f'ENE_{x}'] = (df_consolidado[f'ENE_P_0{x}'] + df_consolidado[f'ENE_F_0{x}'])
        
    else:
        df_consolidado[f'ENE_{x}'] = (df_consolidado[f'ENE_P_{x}'] + df_consolidado[f'ENE_F_{x}'])        


In [ ]:
df_consolidado[["COD_ID","ENE_1"]]

In [ ]:
df_consolidado['ENE_MED'] = df_consolidado.loc[:, 'ENE_1':'ENE_12'].mean(axis=1)
df_consolidado['DEM_MAX'] = df_consolidado.loc[:, 'DEM_1':'DEM_12'].max(axis=1)
df_consolidado['DIC'] = df_consolidado.loc [:,'DIC_01':'DIC_12'].sum(axis=1)
df_consolidado['FIC'] = df_consolidado.loc [:,'FIC_01':'FIC_12'].sum(axis=1)

In [ ]:
#Limpando Colunas desnecessárias

#Excluir Colunas de Energia mês a mês Demanda e Consumo

#Demanda
df_consolidado.drop(['DEM_1', 'DEM_2', 'DEM_3', 'DEM_4', 'DEM_5', 'DEM_6', 'DEM_7', 'DEM_8', 'DEM_9', 'DEM_10','DEM_11', 'DEM_12'],
axis= 1 ,inplace=True)

df_consolidado.drop(['DEM_P_01', 'DEM_P_02', 'DEM_P_03', 'DEM_P_04', 'DEM_P_05', 'DEM_P_06', 'DEM_P_07', 'DEM_P_08', 'DEM_P_09', 
                    'DEM_P_10', 'DEM_P_11', 'DEM_P_12', 'DEM_F_01', 'DEM_F_02', 'DEM_F_03', 'DEM_F_04', 'DEM_F_05', 'DEM_F_06', 
                    'DEM_F_07', 'DEM_F_08', 'DEM_F_09', 'DEM_F_10', 'DEM_F_11', 'DEM_F_12'],axis= 1 ,inplace=True)

#Consumo
df_consolidado.drop(['ENE_1', 'ENE_2', 'ENE_3', 'ENE_4', 'ENE_5', 'ENE_6', 'ENE_7', 'ENE_8', 'ENE_9', 'ENE_10', 'ENE_11',
                     'ENE_12'],axis= 1,inplace=True)

df_consolidado.drop(['ENE_P_01', 'ENE_P_02', 'ENE_P_03', 'ENE_P_04', 'ENE_P_05', 'ENE_P_06', 'ENE_P_07', 'ENE_P_08', 'ENE_P_09',
                     'ENE_P_10', 'ENE_P_11', 'ENE_P_12', 'ENE_F_01', 'ENE_F_02', 'ENE_F_03', 'ENE_F_04', 'ENE_F_05', 'ENE_F_06',
                     'ENE_F_07', 'ENE_F_08', 'ENE_F_09', 'ENE_F_10', 'ENE_F_11', 'ENE_F_12'],axis= 1,inplace=True)

df_consolidado.drop(['DESCR'],axis= 1,inplace=True)



df_consolidado.drop(['DIC_01', 'DIC_02', 'DIC_03', 'DIC_04', 'DIC_05','DIC_06', 'DIC_07', 
'DIC_08', 'DIC_09', 'DIC_10', 'DIC_11', 'DIC_12'],axis= 1,inplace=True)


df_consolidado.drop(['FIC_01', 'FIC_02', 'FIC_03', 'FIC_04', 'FIC_05', 'FIC_06', 'FIC_07',
'FIC_08', 'FIC_09', 'FIC_10', 'FIC_11', 'FIC_12'],axis=1,inplace=True)


In [ ]:
#Adicionando colunas para igualar com MT 
df_consolidado['SEMRED'] = 0
df_consolidado['UNI_TR_S'] = 0

#Renomear a CTAT_CTMT para esse nome tanto no AT e MT
df_consolidado.rename(columns={'CTAT':'CT_AT_MT'},inplace=True)

In [ ]:
df_consolidado.keys()

In [ ]:
# Connect SqlAlchemy
from sqlalchemy import (MetaData,Table,String,
Integer,Float,Column,insert,Numeric,inspect,select,insert)
from common.base import connection_oracle_sqlalchemy

engine= connection_oracle_sqlalchemy()
metadata= MetaData()
connection = engine.connect()
insp = inspect(engine)

In [ ]:
#Criando Tabela Oracle 

bdgd_at = Table('BDGD_AT',
         metadata,
         Column('COD_ID', String(255)),
         Column('DIST', String(255)),
         Column('LGRD', String(255)),
         Column('BRR', String(255)),
         Column('MUNICIPIO', String(255)),       
         Column('ESTADO', String(255)),
         Column('UF', String(255)),
         Column('COD_UF', Numeric),
         Column('MUN', Numeric),
         Column('CEP', String(255)),
         Column('CNAE', String(255)),
         Column('ENE_MED', Float),
         Column('DEM_MAX', Float),
         Column('DEM_CONT',Float),
         Column('LIV', Numeric),
         Column('GRU_TAR',String(255)),                   
         Column('SIT_ATIV', String(255)),
         Column('CLAS_SUB', String(255)),
         Column('DAT_CON', String(255)))

metadata.create_all(engine)

In [15]:
# Loc columns to commit 
df_bdgd_at = df_consolidado\
    .loc[:,['COD_ID','DIST','LGRD','BRR','MUNICIPIO','ESTADO',
            'UF','COD_UF','MUN','CEP','CNAE','ENE_MED',
            'DEM_MAX','DEM_CONT','LIV','GRU_TAR','SIT_ATIV',
            'CLAS_SUB','DAT_CON']].copy()

In [1]:
#Verificando se existe colunas com mais de um types
verifica = [print(colname,types + 'Corrigir')
            if types > 1 else None
            for colname,types in df_bdgd_at.applymap(type).nunique().items()]

NameError: name 'df_bdgd_at' is not defined

In [64]:
#Tranformando List-Dict to SqlAlchemy
values_list =[{'COD_ID':row.COD_ID, 'DIST':row.DIST, 'LGRD':row.LGRD, 'BRR':row.BRR,
  'MUNICIPIO':row.MUNICIPIO, 'ESTADO':row.ESTADO, 'UF':row.UF,
  'COD_UF':row.COD_UF, 'MUN':row.MUN, 'CEP':row.CEP,
  'CNAE':row.CNAE, 'ENE_MED':row.ENE_MED, 'DEM_MAX':row.DEM_MAX,
  'LIV':row.LIV, 'GRU_TAR':row.GRU_TAR,'SIT_ATIV':row.SIT_ATIV,
  'CLAS_SUB':row.CLAS_SUB,'DAT_CON':row.DAT_CON} for row in df_bdgd_at.itertuples()]  

In [68]:
#Commit Oracle - SQL 
stmt = insert(bdgd_at)
results_proxy = connection.execute(stmt,values_list,batcherrors=True)
print(results_proxy.rowcount)

1905
